In [1]:
from pyspark.sql import *
from pyspark.sql.types import *
import os

spark = SparkSession.builder.appName("read csv").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/30 15:59:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [11]:
customer_data_df = spark.read.format("csv")\
                .option("header", "true")\
                .option("inferschema", "false")\
                .option("mode", "PERMISSIVE")\
                .load('/Users/nightfury/Git/Pyspark Practice/data/customers_data.csv')

customer_data_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- customer_phone: string (nullable = true)
 |-- customer_email: string (nullable = true)
 |-- other_customer_details: string (nullable = true)



In [27]:
customer_schema = StructType([
        StructField("customer_id" , IntegerType(), True),
        StructField("customer_name" , StringType(), True),
        StructField("customer_phone" , StringType(), True),
        StructField("customer_email" , StringType(), True),
        StructField("other_customer_details" , StringType(), True),
        StructField("_corrupt_record", StringType(), True)
])

In [34]:
customer_data_df = spark.read.format("csv")\
                .option("header", "true")\
                .schema(customer_schema)\
                .option("badRecordsPath", "/Users/nightfury/Git/Pyspark Practice/bad_records")\
                .load('/Users/nightfury/Git/Pyspark Practice/data/customers_data.csv')

customer_data_df.show(truncate=False)


+-----------+-----------------+----------------------+-------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------+
|customer_id|customer_name    |customer_phone        |customer_email           |other_customer_details                                                                                                                                                           |_corrupt_record                                                                                                                |
+-----------+-----------------+----------------------+-------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------

In [4]:
address_data_df = spark.read.format("csv")\
                       .option("header", True)\
                       .option("inferSchema", True)\
                       .load('/Users/nightfury/Git/Pyspark Practice/data/addresses_data.csv')
address_data_df.show(truncate=False)

+----------+----------------------+--------------------+--------------------+-----------------+------------+---------------------+----------------+
|address_id|line_1_number_building|line_2_number_street|line_3_area_locality|city             |zip_postcode|state_province_county|iso_country_code|
+----------+----------------------+--------------------+--------------------+-----------------+------------+---------------------+----------------+
|1         |594                   |Sheila Highway      |borough             |Michealmouth     |67402       |Alabama              |YE              |
|2         |2388                  |Davis Square        |ton                 |South Melissa    |14998       |Ohio                 |MZ              |
|3         |98786                 |Hernandez Extension |berg                |New Elizabethland|86690       |Oklahoma             |HN              |
|4         |74420                 |Kristina Lock       |view                |Kirstenfurt      |38273       |Cali

In [39]:
customer_data_df.write.format("csv")\
                .option("header", "true")\
                .option("mode", "overwrite")\
                .option("path", '/Users/nightfury/Git/Pyspark Practice/write_data/customers_data.csv')\
                .save()

24/04/29 20:42:24 WARN ParseMode: overwrite is not a valid parse mode. Using PERMISSIVE.


In [41]:
os.listdir("/Users/nightfury/Git/Pyspark Practice/write_data/customers_data.csv")

['.part-00000-3e673f53-b06e-477d-b73f-7d1524f4238d-c000.csv.crc',
 '._SUCCESS.crc',
 'part-00000-3e673f53-b06e-477d-b73f-7d1524f4238d-c000.csv',
 '_SUCCESS']

In [42]:
customer_data_df.repartition(5).write.format("csv")\
                .option("header", "true")\
                .option("mode", "overwrite")\
                .option("path", '/Users/nightfury/Git/Pyspark Practice/write_data/customers_data.csv')\
                .save()

24/04/29 20:45:50 WARN ParseMode: overwrite is not a valid parse mode. Using PERMISSIVE.


In [3]:
product_data_df = spark.read.format("csv")\
                       .option("header", True)\
                       .option("inferSchema", True)\
                       .load('/Users/nightfury/Git/Pyspark Practice/data/products_data.csv')
product_data_df.show(truncate=False)

+----------+-----------------+-----------+------------------+
|product_id|product_type_code|supplier_id|product_price     |
+----------+-----------------+-----------+------------------+
|1         |50               |88         |170.58580589733276|
|2         |58               |84         |46.68355643348467 |
|3         |88               |153        |418.9483581813348 |
|4         |23               |289        |146.95690183009953|
|5         |88               |360        |795.6788412996348 |
|6         |35               |68         |746.3349320376365 |
|7         |81               |185        |653.0086966133251 |
|8         |75               |203        |932.2589100406027 |
|9         |65               |81         |796.1499808446293 |
|10        |9                |346        |450.6431226189617 |
|11        |17               |399        |544.0073240515483 |
|12        |80               |17         |208.10957286664333|
|13        |92               |46         |366.7257252615907 |
|14     

In [5]:
address_data_df.write.format("csv")\
                .option("header", "true")\
                .option("mode", "overwrite")\
                .partitionBy("state_province_county")\
                .option("path", '/Users/nightfury/Git/Pyspark Practice/write_data/address/partition')\
                .save()

24/04/30 16:07:49 WARN ParseMode: overwrite is not a valid parse mode. Using PERMISSIVE.


In [7]:
address_data_df.write.format("csv")\
                .option("header", "true")\
                .option("mode", "overwrite")\
                .bucketBy(3,"state_province_county")\
                .option("path", '/Users/nightfury/Git/Pyspark Practice/write_data/address/buckets')\
                .saveAsTable("address_data_buckets")

24/04/30 22:11:00 WARN ParseMode: overwrite is not a valid parse mode. Using PERMISSIVE.


24/04/30 22:43:17 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1001966 ms exceeds timeout 120000 ms
24/04/30 22:43:17 WARN SparkContext: Killing executors is not supported by current scheduler.
24/04/30 22:47:04 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$